# [lecture_1 notes](Intro_to_Weights_&_Biases.ipynb)
## <span style="color:yellow"> how to initialize a wandb board: the project name, run name </span>
## <span style="color:yellow"> how to add the scalar to the tensorboard </span>
## <span style="color:yellow"> how to create the metric (combine multiple characteristic together)</span>
## <span style="color:yellow"> how to send the alert</span>


- ## wandb.init()
```python 
    wandb.init(
        project="name of the whole project",
        name = "specific one run name" # since there will be multipe run inside the same project name (change the lr)
        config={
            #define the parameters related to model training here such as lr, batch_size, dropout rate...
        }
        notes="give a breif explaination of this run"
        tages = ["help to search"]
    )

    ##code example:
    wandb.init(
    # Set the project where this run will be logged
    project="basic-intro",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"experiment_{run}",
    # Track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    })

```
- ## wandb.log()
    - copy the certain data to the wandb board, and finally after several epock you can see a plot on the board 
```python 
    ## example code:
    wandb.log({"acc": acc, "loss": loss})
    
    # acc and loss are the name of the plot 
```
- ## create metrics and log then to the wandb board 
```python
#define the metrisc
    metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}
#add the metrics
    wandb.log(metrics)
```
- ## dictionary unpackage: add two dictionary together:
```python
    wandb.log({**metrics, **val_metrics})
```

- ## add the alert of wandb (you will receive the email if the alert was triggered)
```python
if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy', # this will be the name of your emial 
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',# this will be the text of that email 
        )
        print('Alert triggered') # this will be print out on your local terminal 
        break
```

# [lecture_2 notes](W&B_Tables_Quickstart.ipynb)
## <span style="color:yellow">teach how to show a table in the wandb board </span>

- ## How to create wandb table 
```python
#step one: call the Table() helper function from wandb 
    #define the name of the column 
    columns=["id", "image", "guess", "truth"]
    for digit in range(10):
      columns.append("score_" + str(digit))

    #call the Table() function 
    test_table = wandb.Table(columns=colums) # here you need to setup the name of each column 
#-------------------------------------------------------------------------------------------------------------
#step two: add the contents to the table 
    test_table.add_data(img_id, wandb.Image(i), p, l, *s) # the data you add should follow the order of the column name

#-------------------------------------------------------------------------------------------------------------
#step three: log the table to the website:
    wandb.log({"test_predictions" : test_table})

#-------------------------------------------------------------------------------------------------------------
#code example:
def log_test_predictions(images, labels, outputs, predicted, test_table, log_counter):
  # obtain confidence scores for all classes
  scores = F.softmax(outputs.data, dim=1)
  log_scores = scores.cpu().numpy()
  log_images = images.cpu().numpy()
  log_labels = labels.cpu().numpy()
  log_preds = predicted.cpu().numpy()
  # adding ids based on the order of the images
  _id = 0
  for i, l, p, s in zip(log_images, log_labels, log_preds, log_scores):
    # add required info to data table:
    # id, image pixels, model's guess, true label, scores for all classes
    img_id = str(_id) + "_" + str(log_counter)
    test_table.add_data(img_id, wandb.Image(i), p, l, *s)
    _id += 1
    if _id == NUM_IMAGES_PER_BATCH:
      break
```

# [lecture 3](Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W&B.ipynb)

## <span style="color:yellow">How to let wandb to automatically search through combinations of hyperparameter values to find values that optimizes your model</span>

## 3 steps to run a hyperparameter sweep:

1. ## **Define the sweep:** we do this by creating a dictionary or a [YAML file](https://docs.wandb.com/library/sweeps/configuration) that specifies the parameters to search through, the search strategy, the optimization metric et all.

    - 1. ### pick a search method:
        - grid 
        - random 
        - Bayesian 
    ```python
    sweep_config = {
    'method': 'random'
    }
    ```
    - 2. ### specify the metric that you want to optimize for
    ```python
    metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

    sweep_config['metric'] = metric
    ```
    - 3. ### specify hyperparameters to search throught 
        - case1: constant value 
        ```python
        parameters_dict.update({
        'epochs': {
            'value': 1}
        })
        ```
        - case2: list out several numbers
        ```python
        parameters_dict = {
            'optimizer': {
                'values': ['adam', 'sgd']
                },
            'fc_layer_size': {
                'values': [128, 256, 512]
                },
            'dropout': {
                'values': [0.3, 0.4, 0.5]
                },
        }
        sweep_config['parameters'] = parameters_dict

        ```
        - case3: give a distribution and set the min, max ...
        ```python
        parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0,
            'max': 0.1
        },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms
            'distribution': 'q_log_uniform_values',
            'q': 8,
            'min': 32,
            'max': 256,
        }
        })
        ```


2. ## **Initialize the sweep:** with one line of code we initialize the sweep and pass in the dictionary of sweep configurations
`sweep_id = wandb.sweep(sweep_config)`
- ### we will use the W&B controller to manage sweeps
- ### the component that actually executes a sweep is known as a sweep agent 
```python 
# you will need to use this sweep_id later
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")
```


3. ## **Run the sweep agent:** also accomplished with one line of code, we call `wandb.agent()` and pass the `sweep_id` to run, along with a function that defines your model architecture and trains it:
`wandb.agent(sweep_id, function=train)`